<img src="../assets/header_notebook.png" />
<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>ESA - Black Sea Deoxygenation Emulator</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# ----------
# Librairies
# ----------
import os
import sys
import cv2
import dawgz
import wandb
import xarray
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
import torch.optim as optim

# Dawgz (jobs //)
from dawgz import job, schedule

# -------------------
# Librairies (Custom)
# -------------------
# Adding path to source folder to load custom modules
sys.path.insert(1, '../src/debs/')
sys.path.insert(1, '../scripts/')

# Loading libraries
from metrics              import *
from tools                import *
from dataset              import *
from dataloader           import *

# -------
# Jupyter
# -------
%matplotlib inline
plt.rcParams.update({'font.size': 13})

# Making sure modules are reloaded when modified
%reload_ext autoreload
%autoreload 2

# Moving to the .py directory
%cd ../src/debs/

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Scripts</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# Training a neural network:
%run training.py    --start_year                 0 \
                    --end_year                   0 \
                    --start_month                0 \
                    --end_month                  1 \
                    --inputs           temperature \
                    --problem       classification \
                    --windows_input              1 \
                    --windows_output             1 \
                    --depth                    200 \
                    --architecture         AVERAGE \
                    --scaling                    1 \
                    --loss_weights             1 1 \
                    --learning_rate          0.001 \
                    --batch_size                16 \
                    --epochs                     1 \
                    --kernel_size                1 \
                    --dawgz                  False

In [ ]:
# Training a neural network:
%run training.py    --start_year                 0 \
                    --end_year                   0 \
                    --start_month                0 \
                    --end_month                  1 \
                    --inputs           temperature \
                    --problem       classification \
                    --windows_input              1 \
                    --windows_output             1 \
                    --depth                    200 \
                    --architecture            UNET \
                    --scaling                    1 \
                    --loss_weights             1 1 \
                    --learning_rate          0.001 \
                    --batch_size                16 \
                    --epochs                     1 \
                    --kernel_size                1 \
                    --dawgz                  False

<hr style="color:#5A7D9F;">
<p align="center">
    <b style="font-size:2vw; color:#5A7D9F; font-weight:bold;">
    <center>Playground</center>
    </b>
</p>
<hr style="color:#5A7D9F;">

In [ ]:
# -----------------
#    Parameters
# -----------------
#
# Time window
month_starting = 0
month_ending   = 1
year_starting  = 0
year_ending    = 0

# Maximum depth observed for oxygen, what is left is masked [m] (Note: To observe only the continental shelf set it to ~200m).
depth_max_oxygen = 1500

# ------------------
#  Loading the data
# ------------------
# Loading the different datasets
Dataset_phy = BlackSea_Dataset(year_start  = year_starting,
                               year_end    = year_ending,
                               month_start = month_starting,
                               month_end   = month_ending,
                               variable    = "grid_T")

Dataset_bio = BlackSea_Dataset(year_start  = year_starting,
                               year_end    = year_ending,
                               month_start = month_starting,
                               month_end   = month_ending,
                               variable    = "ptrc_T")

# Loading the different field values
data_temperature   = Dataset_phy.get_data(variable = "temperature", type = "surface", depth = None)
data_salinity      = Dataset_phy.get_data(variable = "salinity",    type = "surface", depth = None)
data_chlorophyll   = Dataset_bio.get_data(variable = "chlorophyll", type = "surface", depth = None)
data_kshort        = Dataset_bio.get_data(variable = "kshort",      type = "surface", depth = None)
data_klong         = Dataset_bio.get_data(variable = "klong",       type = "surface", depth = None)
data_oxygen        = Dataset_bio.get_data(variable = "oxygen",      type = "bottom" , depth = depth_max_oxygen)

# Loading the black sea masks
bs_mask             = Dataset_phy.get_mask(depth = None)
bs_mask_with_depth  = Dataset_phy.get_mask(depth = depth_max_oxygen)
bs_mask_complete    = get_complete_mask(data_oxygen, bs_mask_with_depth)

# --------------------
#  Preparing the data
# --------------------
# Loading the dataloader
BSD_loader = BlackSea_Dataloader(x = [data_temperature],
                                 y = data_oxygen,
                           bs_mask = bs_mask,
                bs_mask_with_depth = bs_mask_with_depth,
                              mode = "classification",
                        window_inp = 1,
                        window_out = 1)

# Retrieving the datasets
ds_validation = BSD_loader.get_dataloader("validation")
ds_train      = BSD_loader.get_dataloader("train")
ds_test       = BSD_loader.get_dataloader("test")